Tiling 
1. Tile one SAR image
2. Tile one SAR image and the corresponding sea ice chart
3. Tile all SAR images and all corresponding sea ice charts

Removal of NULL values
1. Remove tiled SAR image if it contains null value
2. Remove tiled SAR image and the corresponding sea ice chart if SAR image contains null value
3. Remove all tiled SAR images and their corresponding sea ice chart if SAR images contain nulls

Checking
1. Write a function to validate the outputs at each stage

Notes/to-do: 
1. Allow tile size to be input as a variable
2. Add moving window
3. Add checks to avoid failures / validate the output
4. Write instructions/docs for code
5. Ensure output paths are set correctly
6. Apply the tiling and NULL removal for all files
6. Try to merge the tiling and NILL removal steps into one function (to avoid saving 1000s of tiles before NULLs are removed)
7. Modify code to refer to data in shared OneDrive rather than on my machine

# Step 1: Tile one SAR image

In [136]:
# This worked, creating 1260 files
# CRS, no. of bands, and divisibility into 256x256 are important

import rasterio
from rasterio.windows import Window
import os
from datetime import datetime

def tile_sar_image(input_path, output_folder):
    with rasterio.open(input_path) as src:
        # Get the dimensions of the input image
        width, height = src.width, src.height

        # Extract the date from the filename
        filename = os.path.basename(input_path)
        date_string = filename.split("_")[4]
        date = datetime.strptime(date_string, "%Y%m%dT%H%M%S")
        
        # Calculate the number of tiles needed in the x and y direction
        num_tiles_x = width // 256
        num_tiles_y = height // 256
        
        # Create the window for each tile
        for y in range(num_tiles_y):
            for x in range(num_tiles_x):
                window = Window(x*256, y*256, 256, 256)
                tile = src.read(window=window)
                
                # Save the tile to the output folder
                with rasterio.open(f"{output_folder}/tile_{x}_{y}_{date}.tif", 'w', driver='GTiff', 
                                   width=256, height=256, count=3, crs=src.crs, dtype=tile.dtype,
                                   transform=src.window_transform(window)) as dst:
                    dst.write(tile)

In [137]:
tile_sar_image('/Users/meghanplumridge/Desktop/sea_ice_classification/dual_band_images/S1B_EW_GRDM_1SDH_20180104T224159_20180104T224303_009027_010213_47AC.tif', output_folder='/Users/meghanplumridge/Desktop/sea_ice_classification/Good_SAR_Tile_Output')

# Check output

In [82]:
# Details of input SAR image
SAR_image=rasterio.open('/Users/meghanplumridge/Desktop/sea_ice_classification/dual_band_images/S1B_EW_GRDM_1SDH_20180104T224159_20180104T224303_009027_010213_47AC.tif')
SAR_image_array = SAR_image.read()
SAR_image_metadata = SAR_image.profile
print(SAR_image_metadata)

# Details of output tile
tile_image=rasterio.open('/Users/meghanplumridge/Desktop/sea_ice_classification/tile_0_9.tif')
tile_image_array = tile_image.read()
tile_image_metadata = tile_image.profile
print(tile_image_metadata)

{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 16144, 'height': 5342, 'count': 3, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.0007186522272956172, 0.0, -30.93438512393984,
       0.0, -0.0007186522272956172, -71.44912308995755), 'blockxsize': 256, 'blockysize': 256, 'tiled': True, 'compress': 'lzw', 'interleave': 'pixel'}
{'driver': 'GTiff', 'dtype': 'float32', 'nodata': None, 'width': 256, 'height': 256, 'count': 3, 'crs': CRS.from_epsg(4326), 'transform': Affine(0.0007186522272956172, 0.0, -30.93438512393984,
       0.0, -0.0007186522272956172, -73.10489782164666), 'blockysize': 2, 'tiled': False, 'interleave': 'pixel'}


# Step 2: Tile corresponding sea ice chart

In [118]:
# This worked, creating 1260 files
# Removed count=3

import rasterio
from rasterio.windows import Window

def tile_SIC_image(input_path, output_folder):
    with rasterio.open(input_path) as src:
        # Get the dimensions of the input image
        width, height = src.width, src.height

        # Extract the date from the filename and change to SAR format
        date = input_path.split("/")[-1].split(".")[0]
        formatted_date = "{}-{}-{}".format(date[0:4], date[4:6], date[6:8])
        
        # Calculate the number of tiles needed in the x and y direction
        num_tiles_x = width // 256
        num_tiles_y = height // 256
        
        # Create the window for each tile
        for y in range(num_tiles_y):
            for x in range(num_tiles_x):
                window = Window(x*256, y*256, 256, 256)
                tile = src.read(window=window)
                
                # Save the tile to the output folder
                with rasterio.open(f"{output_folder}/tile_{x}_{y}_{formatted_date}.tif", 'w', driver='GTiff', 
                                   width=256, height=256, count=1, crs=src.crs, dtype=tile.dtype,
                                   transform=src.window_transform(window)) as dst:
                    dst.write(tile)

In [119]:
tile_SIC_image('/Users/meghanplumridge/Desktop/sea_ice_classification/rasterised_shapefiles/20180104.tiff', output_folder='/Users/meghanplumridge/Desktop/sea_ice_classification/')

# Check Output

In [87]:
# Details of input sea ice chart
sea_ice_chart=rasterio.open('/Users/meghanplumridge/Desktop/sea_ice_classification/rasterised_shapefiles/20180104.tiff')
sea_ice_chart_array = sea_ice_chart.read()
sea_ice_chart_metadata = sea_ice_chart.profile
print(sea_ice_chart_metadata)

# Details of one output sea ice chart
sea_ice_chart=rasterio.open('/Users/meghanplumridge/Desktop/sea_ice_classification/SIC_tile_0_1.tif')
sea_ice_chart_array = sea_ice_chart.read()
sea_ice_chart_metadata = sea_ice_chart.profile
print(sea_ice_chart_metadata)

{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': 100.0, 'width': 16144, 'height': 5342, 'count': 1, 'crs': None, 'transform': Affine(0.0007186522272956172, 0.0, -30.93438512393984,
       0.0, 0.0007186522272956172, -75.28816328817074), 'blockysize': 1, 'tiled': False, 'interleave': 'band'}
{'driver': 'GTiff', 'dtype': 'uint8', 'nodata': None, 'width': 256, 'height': 256, 'count': 1, 'crs': None, 'transform': Affine(0.0007186522272956172, 0.0, -30.93438512393984,
       0.0, 0.0007186522272956172, -75.10418831798306), 'blockysize': 32, 'tiled': False, 'interleave': 'band'}


# Step 3: Remove SAR tiles containing NULL values

In [122]:
# Running this on the above examples left me with 1077 files, meaning 1260 - 1077 = 183 tiles were removed.

import os
import rasterio
import numpy as np

# specify the directory containing the .tif files
tile_directory = '/Users/meghanplumridge/Desktop/sea_ice_classification/Good_SAR_Tile_Output/'

# initialize a list to store the names of the files that will be removed
removed_tiles = []

# loop through all .tif files in the directory
for tile in os.listdir(tile_directory):
    if tile.endswith('.tif'):
        file_path = os.path.join(tile_directory, tile)
        
        # open the file using rasterio
        with rasterio.open(file_path) as src:
            data = src.read()
            # check if the file contains any NULL values
            if np.count_nonzero(np.isnan(data)) > 0:
            #if (data == src.nodata).any():
                # if it does, add the file name to the list of removed files
                removed_tiles.append(tile)
                # remove the file
                os.remove(file_path)

# print the list of removed files
print("The following files were removed:")
for file in removed_tiles:
    print(file)
    count(file)

The following files were removed:
tile_61_0_2018-01-04 22:41:59.tif
tile_62_1_2018-01-04 22:41:59.tif
tile_0_18_2018-01-04 22:41:59.tif
tile_0_17_2018-01-04 22:41:59.tif
tile_47_0_2018-01-04 22:41:59.tif
tile_49_2_2018-01-04 22:41:59.tif
tile_8_0_2018-01-04 22:41:59.tif
tile_53_19_2018-01-04 22:41:59.tif
tile_6_2_2018-01-04 22:41:59.tif
tile_51_1_2018-01-04 22:41:59.tif
tile_3_19_2018-01-04 22:41:59.tif
tile_52_0_2018-01-04 22:41:59.tif
tile_59_1_2018-01-04 22:41:59.tif
tile_0_0_2018-01-04 22:41:59.tif
tile_57_3_2018-01-04 22:41:59.tif
tile_49_19_2018-01-04 22:41:59.tif
tile_3_1_2018-01-04 22:41:59.tif
tile_54_2_2018-01-04 22:41:59.tif
tile_53_2_2018-01-04 22:41:59.tif
tile_4_1_2018-01-04 22:41:59.tif
tile_58_16_2018-01-04 22:41:59.tif
tile_61_16_2018-01-04 22:41:59.tif
tile_7_0_2018-01-04 22:41:59.tif
tile_61_19_2018-01-04 22:41:59.tif
tile_55_0_2018-01-04 22:41:59.tif
tile_4_19_2018-01-04 22:41:59.tif
tile_2_3_2018-01-04 22:41:59.tif
tile_61_4_2018-01-04 22:41:59.tif
tile_8_19_2018-0

# Step 4: Remove the corresponding SIC tiles

In [128]:
### Not working yet ###

import os
import numpy as np
import rasterio as rio

# Change SAR filenames to match SIC filenames
filename = "tile_31_19_2018-01-04 22:41:59.tif"
new_filename = filename.split(" ")[0]+".tif"
os.rename(filename, new_filename)SAR_tiles = filename.split(" ")[0]+".tif"

# Remove SAR tiles containing NULL values
def remove_null_tiles(sar_dir, sic_dir):
    removed_tiles = []
    for filename in os.listdir(sar_dir):
        if filename.endswith(".tif"):
            filepath = os.path.join(sar_dir, filename)
            with rio.open(filepath) as src:
                data = src.read(1)
                if np.isnan(data).any():
                    removed_tiles.append(filename)
                    os.remove(filepath)
                    
# Remove corresponding SICs in the filenames match on date
    for filename in os.listdir(sic_dir):
        if filename.endswith(".tif"):
            date = filename.split("_")[-1][:-4]
            if date in [f.split("_")[-1][:-4] for f in removed_tiles]:
                filepath = os.path.join(sic_dir, filename)
                os.remove(filepath)
    
    return removed_tiles

In [133]:
removed_tiles = remove_null_tiles("/Users/meghanplumridge/Desktop/sea_ice_classification/Good_SAR_Tile_Output/", 
                                  "/Users/meghanplumridge/Desktop/sea_ice_classification/Good_SIC_Tile_Output/")

num_removed_files = len(removed_tiles)
print("Number of removed files:", num_removed_files)
print("Removed Tiles: ", removed_tiles)


Number of removed files: 183
Removed Tiles:  ['tile_61_0_2018-01-04 22:41:59.tif', 'tile_62_1_2018-01-04 22:41:59.tif', 'tile_0_18_2018-01-04 22:41:59.tif', 'tile_0_17_2018-01-04 22:41:59.tif', 'tile_47_0_2018-01-04 22:41:59.tif', 'tile_49_2_2018-01-04 22:41:59.tif', 'tile_8_0_2018-01-04 22:41:59.tif', 'tile_53_19_2018-01-04 22:41:59.tif', 'tile_6_2_2018-01-04 22:41:59.tif', 'tile_51_1_2018-01-04 22:41:59.tif', 'tile_3_19_2018-01-04 22:41:59.tif', 'tile_52_0_2018-01-04 22:41:59.tif', 'tile_59_1_2018-01-04 22:41:59.tif', 'tile_0_0_2018-01-04 22:41:59.tif', 'tile_57_3_2018-01-04 22:41:59.tif', 'tile_49_19_2018-01-04 22:41:59.tif', 'tile_3_1_2018-01-04 22:41:59.tif', 'tile_54_2_2018-01-04 22:41:59.tif', 'tile_53_2_2018-01-04 22:41:59.tif', 'tile_4_1_2018-01-04 22:41:59.tif', 'tile_58_16_2018-01-04 22:41:59.tif', 'tile_61_16_2018-01-04 22:41:59.tif', 'tile_7_0_2018-01-04 22:41:59.tif', 'tile_61_19_2018-01-04 22:41:59.tif', 'tile_55_0_2018-01-04 22:41:59.tif', 'tile_4_19_2018-01-04 22:41:59